# Генерация заголовков научных статей:
encoder - предобученный BERT, decoder - GRU

Подключение гугл диска и переход к папке с входными данными и кешом

In [0]:
from google.colab import drive
drive.mount('/content/drive')

root_dir = '.drive/My Drive/'

%cd './drive/My Drive/Colab Notebooks/arxiv_title_generation'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: './drive/My Drive/Colab Notebooks/arxiv_title_generation'
/content/drive/My Drive/Colab Notebooks/arxiv_title_generation
cache  input  output  sample_submission.csv  submission.csv  vocs.pkl


In [0]:
!pip3 install transformers
!pip3 install attrdict

     |████████████████████████████████| 573kB 1.4MB/s 
     |████████████████████████████████| 1.0MB 6.1MB/s 
     |████████████████████████████████| 870kB 9.2MB/s 
     |████████████████████████████████| 3.7MB 10.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=c32a9c199cbeb9427fb0432df7413b2aa06c66d63acb4375ea143c6678e6f511
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import random
import math
import time

from attrdict import AttrDict
import glob
# import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
# from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Dataset
from torch import LongTensor, FloatTensor
# from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from tqdm import trange

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
from IPython.display import clear_output

from transformers import (
    AdamW,
    BertConfig,
    BertModel,
    BertTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup
)

Словарь со всеми исполльзуемыми параметрами

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
args = AttrDict({
    'model_type' : 'bert',
    'device' : torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    'weight_decay' : 0.01,
    'learning_rate' : 5e-5,
    'adam_epsilon' : 1e-8,
    'overwrite_cache' : False,
    'output_dir' : 'output',
    'cache_dir' : 'cache',
    'best_model_dict_file' : 'best_model_dict_bert_gru.pt',
    'train_data_file' : 'input/train.csv',
    'test_data_file' : 'input/test.csv',
    'predicted_data_file' : 'output/predicted_titles.csv',
    'val_size' : 0.01,
    'masked_size' : 0.8,
    'max_abstract_len' : 512,
    'max_title_len' : 64,
    'seed': 42,
    'batch_size' : 32,
    'num_train_epochs' : 14,
    'warmup_steps' : 0,
    'input_dim' : tokenizer.vocab_size,
    'output_dim' : tokenizer.vocab_size,
    'enc_emb_dim' : 512,
    'dec_emb_dim' : 512,
    'enc_hid_dim' : 768,
    'dec_hid_dim' : 256,
    'enc_dropout' : 0.1,
    'att_dropout' : 0.3,
    'dec_dropout' : 0.8,
    'pad_idx' : tokenizer.pad_token_id,
    'eos_idx' : tokenizer.sep_token_id,
    'sos_idx' : tokenizer.cls_token_id
})

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)
if torch.cuda.is_available:
    torch.cuda.manual_seed_all(args.seed)
torch.backends.cudnn.deterministic = True

# Подготовка данных для обучения

In [0]:
tokenizer.vocab_size

30522

Не использую TensorDataSet, так как необходимы titles переменной длины. Для этого определяю свой DataSet

In [0]:
def my_collate(batch):
    src = []
    trg = []
    for item in batch:
        src_item, trg_item = item
        src.append(src_item)
        trg.append(trg_item)
    max_trg_len = max(map(len, trg))
    for trg_item in trg:
        for i in range(len(trg_item), max_trg_len):
            trg_item.append(args.pad_idx)
    return (LongTensor(src), LongTensor(trg))


class SimpleDataSet(Dataset):
    def __init__(self, *objects):
        self.samples = []
        for iterable in zip(*objects):
            self.samples.append(iterable)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

Берем данные из кеша, если возможно

In [0]:
# функция кодирования текста статьи
def encode_abstract(x:str) -> list:
    x = x.replace('. ', '. {} '.format(tokenizer.sep_token))
    return tokenizer.encode(x, add_special_tokens=True, max_length=args.max_abstract_len, pad_to_max_length=True)

# функция кодирования заголовка статьи
def encode_title(x:str) -> list:
    return tokenizer.encode(x, add_special_tokens=True, max_length=args.max_title_len)

# загрузка данных для обучения
def load_train_data(tokenizer:PreTrainedTokenizer) -> Tuple[DataLoader, DataLoader]:
    assert os.path.isfile(args.train_data_file)

    filedir, filename = os.path.split(args.train_data_file)
    cached_abstract_file = os.path.join(
        args.cache_dir, filedir, args.model_type + "_cached_" + str(args.max_abstract_len) + "_" + filename.split('.')[0] + '_abstract.list'
    )
    cached_title_file = os.path.join(
        args.cache_dir, filedir, args.model_type + "_cached_" + str(args.max_title_len) + "_" + filename.split('.')[0] + '_title.list'
    )

    if os.path.exists(cached_abstract_file) and not args.overwrite_cache:
        print("Loading features from cached file {}, {}".format(cached_abstract_file, cached_title_file))
        with open(cached_abstract_file, "rb") as fp:
            abstracts = pickle.load(fp)
        with open(cached_title_file, "rb") as fp:
            titles = pickle.load(fp)

    else:
        print("Creating features from dataset file {}".format(filename))

        data = pd.read_csv(args.train_data_file, encoding='utf8')
        abstracts = []
        for abstract in tqdm(data.abstract.values, desc="Encode abstacts"):
            abstracts.append(encode_abstract(abstract))
        titles = []
        for title in tqdm(data.title.values, desc="Encode titles"):
            titles.append(encode_title(title))

        os.makedirs(os.path.join(args.cache_dir, filedir), exist_ok=True)
        with open(cached_abstract_file, "wb") as fp:
            pickle.dump(abstracts, fp)
        with open(cached_title_file, "wb") as fp:
            pickle.dump(titles, fp)

        print("Features has been successfuly created")

    train_abstracts, val_abstracts, train_titles, val_titles = train_test_split(
        abstracts,
        titles,
        random_state=args.seed,
        test_size=args.val_size
    )

    train_data = SimpleDataSet(train_abstracts, train_titles)
    train_dataloader = DataLoader(
        train_data,
        sampler=RandomSampler(train_data),
        batch_size=args.batch_size,
        collate_fn=my_collate
    )
    validation_data = SimpleDataSet(val_abstracts, val_titles)
    validation_dataloader = DataLoader(
        validation_data,
        sampler=SequentialSampler(validation_data),
        batch_size=args.batch_size,
        collate_fn=my_collate
    )

    return train_dataloader, validation_dataloader

# Seq2Seq модель

In [0]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

In [0]:
# класс, отвечающий за обработку основного текста статьи
class Encoder(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        config = BertConfig.from_pretrained("bert-base-uncased")
        self.model = BertModel(config)
        self.dropout = nn.Dropout(dropout)
        # self.fc1 = nn.Linear(enc_hid_dim, dec_hid_dim)
        self.fc2 = nn.Linear(enc_hid_dim, dec_hid_dim)
        self.act = nn.Tanh()
        # for param in model.parameters():
        #     param.requires_grad = False

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, **kwargs):
        if attention_mask is None:
            attention_mask = torch.ones_like(input_ids)
            idx_pad_tokens = input_ids == args.pad_idx
            attention_mask[idx_pad_tokens] = 0
        # attention_mask = self.dropout(attention_mask)
        # self.eval()
        # with torch.no_grad():
        outputs, hidden = self.model(
            input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, **kwargs
        )

        #outputs = [batch size, sent len, enc hid dim]
        #hidden = [batch size, enc hid dim]

        # outputs = self.act(self.fc1(outputs))
        hidden = self.act(self.fc2(hidden))

        #outputs = [batch size, sent len, enc hid dim]
        #hidden = [batch size, dec hid dim]

        outputs =  self.dropout(outputs)
        hidden = self.dropout(hidden)
        
        return outputs, hidden

In [0]:
# класс, отвечающий за взамосвязь энкодера и декодера
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.attn = nn.Linear(enc_hid_dim + dec_hid_dim, (enc_hid_dim + 2 * dec_hid_dim) // 2)
        self.attn2 = nn.Linear((enc_hid_dim + 2 * dec_hid_dim) // 2, dec_hid_dim)
        self.act = nn.Tanh()
        self.v = nn.Parameter(torch.rand(dec_hid_dim))
        self.dropout = nn.Dropout(dropout)
        self.apply(init_weights)
        
    def forward(self, hidden, encoder_outputs, mask):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [batch size, src sent len, dec hid dim]
        #mask = [batch size, src sent len]
        
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        #repeat encoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        # encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src sent len, dec hid dim]
        #encoder_outputs = [batch size, src sent len, enc hid dim]
        
        energy = self.act(self.attn(torch.cat((hidden, encoder_outputs), dim = 2)))
        energy = self.dropout(energy)
        energy = self.act(self.attn2(energy))
        # energy = torch.tanh(self.attn(encoder_outputs)) 
        #energy = [batch size, src sent len, dec hid dim]
                
        energy = energy.permute(0, 2, 1)
        
        #energy = [batch size, dec hid dim, src sent len]
        
        #v = [dec hid dim]
        
        v = self.v.repeat(batch_size, 1).unsqueeze(1)
        
        #v = [batch size, 1, dec hid dim]
            
        attention = torch.bmm(v, energy).squeeze(1)

        #attention = [batch size, src sent len]
        
        attention = attention.masked_fill(mask == 0, -1e10)
        
        return F.softmax(attention, dim=-1)

In [0]:
# класс, отвечающий за генерацию заголовка
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU(enc_hid_dim + emb_dim, dec_hid_dim)
        
        self.out = nn.Linear(enc_hid_dim + dec_hid_dim + emb_dim, output_dim)
        self.act = nn.Tanh()
        
        self.dropout = nn.Dropout(dropout)

        self.apply(init_weights)
        
    def forward(self, input, hidden, encoder_outputs, mask):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [batch size, src sent len, dec hid dim]
        #mask = [batch size, src sent len]
        
        input = input.unsqueeze(-1)
        #input = [batch size, 1]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [batch size, 1, emb dim]
        
        a = self.attention(hidden, encoder_outputs, mask)
                
        #a = [batch size, src sent len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src sent len]
        
        # encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src sent len, enc hid dim]
        
        weighted = torch.bmm(a, encoder_outputs)
        # print(weighted.shape)
        
        #weighted = [batch size, 1, enc hid dim]
        
        # weighted = weighted.permute(1, 0, 2)
        # print(weighted.shape)
        
        #weighted = [batch size, 1, enc hid dim]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2).transpose(0, 1)
        # print(rnn_input.shape)
        #rnn_input = [1, batch size, (enc hid dim) + emb dim]
            
        output, hidden = self.rnn(rnn_input)
        
        #output = [sent len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #sent len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()

        output = output.transpose(0, 1)
        hidden = hidden.transpose(0, 1)

        #output = [batch size, 1, dec hid dim]
        #hidden = [batch size, 1, dec hid dim]
        
        embedded = embedded.squeeze(1)
        output = output.squeeze(1)
        weighted = weighted.squeeze(1)
        hidden = hidden.squeeze(1)
        a = a.squeeze(1)
        
        output = self.act(self.out(torch.cat((output, weighted, embedded), dim=-1)))
        
        #output = [batch size, output dim]
        
        return output, hidden, a

In [0]:
# класс, соединяющий все части итоговой архитектуры воедино
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, pad_idx, sos_idx, eos_idx, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.pad_idx = pad_idx
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.device = device
        # freeze encoder
        for param in self.encoder.model.parameters():
            param.requires_grad = False
        
    def create_mask(self, src):
        mask = (src != self.pad_idx)#.permute(1, 0)
        return mask
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        # print(src.shape)
        #src = [batch size, src sent len]
        #trg = [batch size, trg sent len]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        if trg is None:
            assert teacher_forcing_ratio == 0, "Must be zero during inference"
            inference = True
            trg = torch.zeros((src.shape[0], args.max_title_len)).long().fill_(self.sos_idx).to(src.device)
        else:
            inference = False
            
        batch_size = src.shape[0]
        max_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(batch_size, max_len, trg_vocab_size).to(self.device)
        for batch in outputs:
            batch[0][args.sos_idx] = float('inf')
        
        #tensor to store attention
        attentions = torch.zeros(batch_size, max_len, src.shape[1]).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[:,0]
        
        mask = self.create_mask(src)#.permute(1, 0)
                
        #mask = [batch size, src sent len]
                
        for t in range(1, max_len):
            
            #insert input token embedding, previous hidden state, all encoder hidden states 
            # and mask
            #receive output tensor (predictions), new hidden state and attention tensor
            output, hidden, attention = self.decoder(input, hidden, encoder_outputs, mask)
            
            #place predictions in a tensor holding predictions for each token
            outputs[:, t] = output
            
            #place attentions in a tensor holding attention value for each input token
            attentions[:, t] = attention
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[:,t] if teacher_force else top1
            
            #if doing inference and next token/prediction is an eos token then stop
            if inference and input.item() == self.eos_idx:
                return outputs[:, :t+1], attentions[:, :t+1]
            
        return outputs, attentions

    def predict(self, src, beam_size=10):
        vocab_size = args.output_dim
        alive_seq = torch.full([beam_size, 1], self.sos_idx, dtype=torch.long, device=self.device)
        
        src = src.unsqueeze(0).repeat(beam_size, 1)
        encoder_outputs, hidden = self.encoder(src)

        mask = self.create_mask(src)

        # for step in range(args.max_title_len):
        input = alive_seq[:, -1]

        outputs, hidden, attention = self.decoder(input, hidden, encoder_outputs, mask) 

        curr_scores = outputs.reshape(-1, beam_size * vocab_size)
        print(curr_scores.shape)
        topk_scores, topk_ids = curr_scores.topk(beam_size, dim=-1)
        topk_beam_index = topk_ids.div(vocab_size)
        print(topk_beam_index.shape)
        topk_ids = topk_ids.fmod(vocab_size)
        print(topk_ids.shape)

        alive_seq = torch.cat([alive_seq.index_select(0, topk_beam_index), topk_ids.view(-1, 1)], -1)


In [0]:
# INPUT_DIM = len(TEXT.vocab) #40710
# OUTPUT_DIM = len(TEXT.vocab) #40710
# ENC_EMB_DIM = 128
# DEC_EMB_DIM = 128
# ENC_HID_DIM = 64
# DEC_HID_DIM = 64
# ENC_DROPOUT = 0.8
# DEC_DROPOUT = 0.8
# PAD_IDX = TEXT.vocab.stoi['<pad>']
# SOS_IDX = TEXT.vocab.stoi['<sos>']
# EOS_IDX = TEXT.vocab.stoi['<eos>']

attn = Attention(args.enc_hid_dim, args.dec_hid_dim, args.att_dropout)
enc = Encoder(args.enc_hid_dim, args.dec_hid_dim, args.enc_dropout)
dec = Decoder(args.output_dim, args.dec_emb_dim, args.enc_hid_dim, args.dec_hid_dim, args.dec_dropout, attn)

model = Seq2Seq(enc, dec, args.pad_idx, args.sos_idx, args.eos_idx, args.device).to(args.device)

In [0]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 64,737,978 trainable parameters


### Обучение модели

In [0]:
optimizer = AdamW(model.parameters())

In [0]:
def _is_long(x):
    if hasattr(x, 'data'):
        x = x.data
    return isinstance(x, torch.LongTensor) or isinstance(x, torch.cuda.LongTensor)


def cross_entropy(inputs, target, weight=None, ignore_index=-100, reduction='mean',
                  smooth_eps=None, smooth_dist=None, from_logits=True):
    """cross entropy loss, with support for target distributions and label smoothing https://arxiv.org/abs/1512.00567"""
    smooth_eps = smooth_eps or 0

    # ordinary log-liklihood - use cross_entropy from nn
    if _is_long(target) and smooth_eps == 0:
        if from_logits:
            return F.cross_entropy(inputs, target, weight, ignore_index=ignore_index, reduction=reduction)
        else:
            return F.nll_loss(inputs, target, weight, ignore_index=ignore_index, reduction=reduction)

    if from_logits:
        # log-softmax of inputs
        lsm = F.log_softmax(inputs, dim=-1)
    else:
        lsm = inputs

    masked_indices = None
    num_classes = inputs.size(-1)

    if _is_long(target) and ignore_index >= 0:
        masked_indices = target.eq(ignore_index)

    if smooth_eps > 0 and smooth_dist is not None:
        if _is_long(target):
            target = onehot(target, num_classes).type_as(inputs)
        if smooth_dist.dim() < target.dim():
            smooth_dist = smooth_dist.unsqueeze(0)
        target.lerp_(smooth_dist, smooth_eps)

    if weight is not None:
        lsm = lsm * weight.unsqueeze(0)

    if _is_long(target):
        eps_sum = smooth_eps / num_classes
        eps_nll = 1. - eps_sum - smooth_eps
        likelihood = lsm.gather(dim=-1, index=target.unsqueeze(-1)).squeeze(-1)
        loss = -(eps_nll * likelihood + eps_sum * lsm.sum(-1))
    else:
        loss = -(target * lsm).sum(-1)

    if masked_indices is not None:
        loss.masked_fill_(masked_indices, 0)

    if reduction == 'sum':
        loss = loss.sum()
    elif reduction == 'mean':
        if masked_indices is None:
            loss = loss.mean()
        else:
            loss = loss.sum() / float(loss.size(0) - masked_indices.sum())

    return loss


class CrossEntropyLoss(nn.CrossEntropyLoss):
    """CrossEntropyLoss - with ability to recieve distrbution as targets, and optional label smoothing"""

    def __init__(self, weight=None, ignore_index=-100, reduction='mean', smooth_eps=None, smooth_dist=None, from_logits=True):
        super(CrossEntropyLoss, self).__init__(weight=weight,
                                               ignore_index=ignore_index, reduction=reduction)
        self.smooth_eps = smooth_eps
        self.smooth_dist = smooth_dist
        self.from_logits = from_logits

    def forward(self, input, target, smooth_dist=None):
        if smooth_dist is None:
            smooth_dist = self.smooth_dist
        return cross_entropy(input, target, weight=self.weight, ignore_index=self.ignore_index,
                             reduction=self.reduction, smooth_eps=self.smooth_eps,
                             smooth_dist=smooth_dist, from_logits=self.from_logits)
        
criterion = CrossEntropyLoss(ignore_index = args.pad_idx, smooth_eps=0.05)

In [0]:
def train(model, iterator, optimizer, criterion, clip, train_history=None, valid_history=None):
    
    model.train()
    # print(type(iterator))
    tqdm_iterator = tqdm(iterator, desc="Train iteration")
    epoch_loss = 0
    history = []
    for i, batch in enumerate(tqdm_iterator):
        
        src, trg = tuple(t.to(args.device) for t in batch)

        optimizer.zero_grad()
        
        output, attetion = model(src, trg)
        
        #trg = [batch size, trg sent len]
        #output = [trg sent len, batch size, output dim]
        # print(output.shape)
        # print(trg.shape)
        
        output = output[:, 1:].reshape(-1, output.shape[-1])
        trg = trg[:, 1:].reshape(-1)
        
        # print(output.shape)
        # print(trg.shape)
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
        history.append(loss.cpu().data.numpy())
        # if (i+1)%10==0:
        #     fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

        #     clear_output(True)
        #     ax[0].plot(history, label='train loss')
        #     ax[0].set_xlabel('Batch')
        #     ax[0].set_title('Train loss')
        #     if train_history is not None:
        #         ax[1].plot(train_history, label='general train history')
        #         ax[1].set_xlabel('Epoch')
        #     if valid_history is not None:
        #         ax[1].plot(valid_history, label='general valid history')
        #     plt.legend()
            
        #     plt.show()
        
    return epoch_loss / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    tqdm_iterator = tqdm(iterator, desc="Validation iteration")
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(tqdm_iterator):

            src, trg = tuple(t.to(args.device) for t in batch)

            output, attention = model(src, trg, 0) #turn off teacher forcing

            #trg = [batch size, trg sent len]
            #output = [trg sent len, batch size, output dim]

            output = output[:, 1:].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [0]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
# to get results like in leaderboard train the model
# for 10 epochs on the whole dataset (135000 samples instead of 30000)

N_EPOCHS = args.num_train_epochs
CLIP = 1

train_history = []
valid_history = []

best_valid_loss = float('inf')
# best_valid_loss = 6.317
train_dataloader, valid_dataloader = load_train_data(tokenizer)

for epoch in range(args.num_train_epochs):
    print("Epoch {:d}/{:d}".format(epoch+1, args.num_train_epochs))
    start_time = time.time()
    
    train_loss = train(model, train_dataloader, optimizer, criterion, CLIP, train_history, valid_history)
    valid_loss = evaluate(model, valid_dataloader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time) 
    
    if valid_loss < best_valid_loss:
        model_dict_path = os.path.join(args.cache_dir, args.best_model_dict_file)
        torch.save(model.state_dict(), model_dict_path)
        best_valid_loss = valid_loss
        
        
    train_history.append(train_loss)
    valid_history.append(valid_loss)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

_, ax = plt.subplots(figsize=(12, 8))
ax.set_xlabel('Epoch')
ax.plot(train_history, label='train history')
ax.plot(valid_history, label='valid history')
plt.show()

Loading features from cached file cache/input/bert_cached_512_train_abstract.list, cache/input/bert_cached_64_train_title.list
Epoch 1/14



Epoch: 01 | Time: 95m 7s
	Train Loss: 9.085 | Train PPL: 8822.166
	 Val. Loss: 9.081 |  Val. PPL: 8788.506
Epoch 2/14



Epoch: 02 | Time: 136m 29s
	Train Loss: 9.089 | Train PPL: 8859.470
	 Val. Loss: 9.089 |  Val. PPL: 8857.160
Epoch 3/14



Epoch: 03 | Time: 153m 27s
	Train Loss: 9.093 | Train PPL: 8895.617
	 Val. Loss: 9.090 |  Val. PPL: 8867.304
Epoch 4/14



Epoch: 04 | Time: 153m 36s
	Train Loss: 9.096 | Train PPL: 8919.324
	 Val. Loss: 9.093 |  Val. PPL: 8888.669
Epoch 5/14



Epoch: 05 | Time: 153m 54s
	Train Loss: 9.098 | Train PPL: 8934.333
	 Val. Loss: 9.093 |  Val. PPL: 8895.537
Epoch 6/14



Epoch: 06 | Time: 153m 57s
	Train Loss: 9.099 | Train PPL: 8943.320
	 Val. Loss: 9.094 |  Val. PPL: 8897.975
Epoch 7/14



Epoch: 07 | Time: 154m 4s
	Train Loss: 9.100 | Train PPL: 8952.152
	 Val. Loss: 9.095 |  Val. PPL: 8913.928
Epoch 8/14



Epoch: 08 | Time: 153m 40s
	Train Loss: 9.101 | Train PPL: 8964.355
	 Val. Loss: 9.097 |  Val. PPL: 8931.048
Epoch 9/14


Buffered data was truncated after reaching the output size limit.

In [0]:
model_dict_path = os.path.join(args.cache_dir, args.best_model_dict_file)
model.load_state_dict(torch.load(model_dict_path, map_location=args.device))

In [0]:
_, valid_dataloader = load_train_data(tokenizer)
test_loss = evaluate(model, valid_dataloader, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

### Генерация заголовков

In [0]:
# def beam_search(data, k, eos_idx):
#     data -= data.min()
#     data /= data.max()
#     best_indexes = data[0].sort(dim=0, descending=True).indices[:3].tolist()
#     sequences = [[[index], score if score != 0 else 0.0001] for index, score in zip(best_indexes, data[0][best_indexes].tolist())]
#     # print(sequences)
#     # walk over each step in sequence
#     for row in data[1:]:
#         all_candidates = sequences.copy()
#         # expand each current candidate
#         for i in range(len(sequences)):
#             seq, score = sequences[i]
#             if seq[-1] != eos_idx: 
#                 for j in range(len(row)):
#                     delta = row[j].item() if row[j].item() != 0 else 0.0001
#                     candidate = [seq + [j], (score * delta)**(1/(len(seq)+1)) ]
#                     all_candidates.append(candidate)
#         # order all candidates by score
#         # print(all_candidates)
#         ordered = sorted(all_candidates, key=lambda tup:tup[1], reverse=True)
#         # select k best
#         sequences = ordered[:k]
#     return sequences[0][0]

In [0]:
def decode(translation_tensor: LongTensor, skip_special_tokens=False) -> str:
    return tokenizer.decode(translation_tensor, skip_special_tokens=skip_special_tokens)


def translate_abstract(model, abstract, skip_special_tokens=False):
    model.eval()
    encoded_abstract = encode_abstract(abstract)
    tensor = LongTensor(encoded_abstract).to(args.device)
    # print(tensor.shape)
    translation_tensor_logits, attention = model.predict(tensor, 10)
    translation_tensor = translation_tensor_logits.squeeze(0).argmax(-1)
    translation = decode(translation_tensor, skip_special_tokens)
    attention.squeeze_(0)
    return translation, attention, translation_tensor

In [0]:
def display_attention(sentence, translation_tensor, attention):
    
    fig = plt.figure(figsize=(60,100))
    ax = fig.add_subplot(111)
    
    attention = attention.squeeze(1).cpu().detach().numpy().T
    
    cax = ax.matshow(attention, cmap='bone')
   
    ax.tick_params(labelsize=12)
    ax.set_yticklabels(['']+tokenizer.convert_ids_to_tokens(encode_abstract(sentence)))
    ax.set_xticklabels(['']+tokenizer.convert_ids_to_tokens(translation_tensor), rotation=80)

    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()
    plt.close()

In [0]:
example_idx = 100

example = pd.read_csv(args.train_data_file).iloc[example_idx]
src = example.abstract
trg = example.title

print(f'src = {src}')
print(f'trg = {trg}')

In [0]:
translation, attention, translation_tensor = translate_abstract(model, src, skip_special_tokens=False)

print(f'predicted trg = {translation}')

In [0]:
print(translation_tensor.shape)

In [0]:
display_attention(src, translation_tensor, attention)

In [0]:
data = pd.read_csv(args.train_data_file)

for example_idx in range(100):
    src = data.iloc[example_idx].abstract
    trg = data.iloc[example_idx].title
    translation, attention, _ = translate_abstract(model, src, skip_special_tokens=True)

    print('Оригинальный заголовок: ', trg)
    print('Предсказанный заголовок: ', translation)
    print('-----------------------------------')

### Считаем BLEU

In [0]:
import nltk

n_gram_weights = [0.3334, 0.3333, 0.3333]

In [0]:
test_len = len(test_data)

In [0]:
original_texts = []
generated_texts = []
macro_bleu = 0

for example_idx in range(test_len):
    src = vars(test_data.examples[example_idx])['src']
    trg = vars(test_data.examples[example_idx])['trg']
    translation, _ = translate_sentence(model, src)

    original_texts.append(trg)
    generated_texts.append(translation)

    bleu_score = nltk.translate.bleu_score.sentence_bleu(
        [trg],
        translation,
        weights = n_gram_weights
    )    
    macro_bleu += bleu_score

macro_bleu /= test_len

In [0]:
# averaging sentence-level BLEU (i.e. macro-average precision)
print('Macro-average BLEU (LSTM): {0:.5f}'.format(macro_bleu))

### Делаем submission в Kaggle

Генерация заголовков для тестовых данных:

In [0]:
submission_data = pd.read_csv(args.test_data_file)
abstracts = submission_data['abstract'].values
titles = []
for abstract in tqdm(abstracts, desc="Test iteration"):
    title, _, _ = translate_abstract(model, abstract, skip_special_tokens=True)
    titles.append(title)

Записываем полученные заголовки в файл формата `<abstract>,<title>`:

In [0]:
submission_df = pd.DataFrame({'abstract': abstracts, 'title': titles})
submission_df.to_csv(args.predicted_data_file, index=False)

С помощью функции `generate_csv` приводим файл `submission_prediction.csv` в формат, необходимый для посылки в соревнование на Kaggle:

In [0]:
import string
from nltk.util import ngrams
import numpy as np
import pandas as pd
import pickle


def generate_csv(input_file=args.predicted_data_file,
                 output_file='submission.csv',
                 voc_file='vocs.pkl'):
    '''
    Generates file in format required for submitting result to Kaggle
    
    Parameters:
        input_file (str) : path to csv file with your predicted titles.
                           Should have two fields: abstract and title
        output_file (str) : path to output submission file
        voc_file (str) : path to voc.pkl file
    '''
    data = pd.read_csv(input_file)
    with open(voc_file, 'rb') as voc_file:
        vocs = pickle.load(voc_file)

    with open(output_file, 'w') as res_file:
        res_file.write('Id,Predict\n')
        
    output_idx = 0
    for row_idx, row in data.iterrows():
        trg = row['title']
        trg = trg.translate(str.maketrans('', '', string.punctuation)).lower().split()
        trg.extend(['_'.join(ngram) for ngram in list(ngrams(trg, 2)) + list(ngrams(trg, 3))])
        
        VOCAB_stoi = vocs[row_idx]
        trg_intersection = set(VOCAB_stoi.keys()).intersection(set(trg))
        trg_vec = np.zeros(len(VOCAB_stoi))    

        for word in trg_intersection:
            trg_vec[VOCAB_stoi[word]] = 1

        with open(output_file, 'a') as res_file:
            for is_word in trg_vec:
                res_file.write('{0},{1}\n'.format(output_idx, int(is_word)))
                output_idx += 1


generate_csv()